In [38]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [39]:
%cd "/content/drive/My Drive/Campcode Challenge/challenge2/data/"
%ls

/content/drive/My Drive/Campcode Challenge/challenge2/data
labelled.csv  train/  val/


In [40]:
train_data_dir = 'train'
validation_data_dir = 'val'

img_width, img_height = 100, 100
batch_size_train = 64
batch_size_test = 32

In [82]:
num_train = 2659
num_val = 2658

In [83]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras import optimizers

In [84]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [106]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size_train,
    class_mode='categorical', 
    color_mode='grayscale')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size_test,
    class_mode='categorical',
    color_mode='grayscale')

Found 2659 images belonging to 6 classes.
Found 2658 images belonging to 6 classes.


In [107]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(img_width, img_height, 1)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

In [112]:

sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

In [113]:
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [114]:
train_history_adam = model.fit_generator(
    train_generator,
    steps_per_epoch=num_train/batch_size_train,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=num_val/batch_size_test)

Epoch 1/10
42/41 [==============================] - 50s 1s/step - loss: 1.3086 - accuracy: 0.5965 - val_loss: 0.8759 - val_accuracy: 0.8281
Epoch 2/10
42/41 [==============================] - 51s 1s/step - loss: 1.3086 - accuracy: 0.5965 - val_loss: 0.8668 - val_accuracy: 0.8281
Epoch 3/10
42/41 [==============================] - 50s 1s/step - loss: 1.3059 - accuracy: 0.5965 - val_loss: 0.8753 - val_accuracy: 0.8281
Epoch 4/10
42/41 [==============================] - 50s 1s/step - loss: 1.3061 - accuracy: 0.5965 - val_loss: 0.8689 - val_accuracy: 0.8281
Epoch 5/10
42/41 [==============================] - 50s 1s/step - loss: 1.3095 - accuracy: 0.5965 - val_loss: 0.8738 - val_accuracy: 0.8281
Epoch 6/10
42/41 [==============================] - 50s 1s/step - loss: 1.3066 - accuracy: 0.5965 - val_loss: 0.8717 - val_accuracy: 0.8281
Epoch 7/10
32/41 [======================>.......] - ETA: 8s - loss: 1.3310 - accuracy: 0.5850

KeyboardInterrupt: ignored

In [111]:
model.save_weights('\challenge2_weight.h5')
model.save('\challenge2_model.h5')

In [115]:
model = keras.models.load_model('\challenge2_model.h5')

In [117]:
model.evaluate(validation_generator)

84/84 [==============================] - 15s 183ms/step - loss: 0.8808 - accuracy: 0.8281


[0.8808320760726929, 0.8280662298202515]